# CNN para Pronóstico de Series Temporales y Discusión de Artículo Científico

Este notebook muestra una implementación de una Red Neuronal Convolucional (CNN) aplicada al pronóstico de series temporales sintéticas. Además, se presenta y discute un artículo científico relacionado con la aplicación de CNN en el ámbito de la predicción de la demanda eléctrica:

> **Referencia**: Deng, J., Yang, G., Li, K., & Li, K. (2020). *An improved deep learning approach for short-term load forecasting with attention mechanism*. IEEE Transactions on Smart Grid, 11(2), 1427-1436.

En dicho artículo, los autores investigan cómo una arquitectura basada en CNN, complementada con un mecanismo de atención, mejora el desempeño en el pronóstico de la carga eléctrica a corto plazo. El uso de la CNN permite capturar patrones locales de la serie temporal, mientras que el mecanismo de atención refuerza la relevancia de características específicas al momento de generar las predicciones. Esto se traduce en una reducción del error y en una mayor estabilidad de los pronósticos.

En este notebook, se explicarán los pasos básicos para construir y entrenar una CNN para la predicción de valores futuros de una serie temporal sintética. Al final, se presentará una comparación con las conclusiones del artículo de Deng et al. (2020), resaltando las ventajas de las CNN y el potencial de complementarlas con mecanismos de atención.

## Objetivos del Notebook

- Implementar una Red Neuronal Convolucional (CNN) para el pronóstico de una serie temporal sintética.
- Evaluar el desempeño del modelo en términos de error de predicción.
- Discutir las ventajas de usar CNN en series temporales, relacionándolo con el artículo de Deng et al. (2020).

## Importación de Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 1. Generación de Datos Sintéticos

Se genera una serie temporal a partir de una onda senoidal con ruido aleatorio, simulando datos que podrían representar un proceso periódico con variaciones propias de una señal real.

In [ ]:
np.random.seed(0)
t = np.arange(0, 100, 0.1)

# Onda senoidal base
sinusoid = np.sin(0.1 * t)

# Ruido aleatorio
noise = np.random.normal(0, 0.1, len(t))

# Serie temporal final
time_series = sinusoid + noise

# Visualización de la serie
plt.figure(figsize=(12, 5))
plt.plot(t, time_series)
plt.title('Serie Temporal Sintética (Seno + Ruido)')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.grid(True)
plt.show()

## 2. Preparación de Datos

Se crean muestras de longitud fija (ventanas deslizantes) para pronosticar un paso futuro. Aquí, empleamos 10 pasos históricos (n_steps=10) para predecir el siguiente valor en la serie temporal.

En la práctica, es recomendable explorar enfoques como validación cruzada para series temporales o expandir esta ventana de predicción a horizontes múltiples si se requiere pronosticar más de un paso a futuro.

In [ ]:
n_steps = 10
X, y = [], []
for i in range(len(time_series) - n_steps):
    X.append(time_series[i:i+n_steps])
    y.append(time_series[i+n_steps])

X = np.array(X)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Conv1D espera (batch, steps, canales)
y = np.array(y)

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Forma de X_train: {X_train.shape}')
print(f'Forma de y_train: {y_train.shape}')
print(f'Forma de X_test : {X_test.shape}')
print(f'Forma de y_test : {y_test.shape}')

## 3. Construcción de la CNN

Se emplea una arquitectura sencilla de tipo **Conv1D** + **MaxPooling1D**, seguida de capas densas para la regresión. El modelo se compila con la función de pérdida MSE y el optimizador Adam.

In [ ]:
# Definición del modelo
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

# Compilación
model.compile(optimizer='adam', loss='mse')
model.summary()

## 4. Entrenamiento

Entrenamos el modelo durante 50 épocas, guardando el registro de la pérdida de entrenamiento y validación para monitorear el progreso.

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

# Gráfica del historial de entrenamiento
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Historial de Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.grid(True)
plt.show()

## 5. Evaluación del Modelo

Se calcula el error en el conjunto de prueba y se comparan los valores reales con las predicciones. Como métrica se utiliza el RMSE, una de las más comunes para tareas de regresión en series temporales.

In [ ]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE en el conjunto de prueba: {rmse:.4f}')

plt.figure(figsize=(12, 5))
plt.plot(y_test, label='Valores Reales', marker='o')
plt.plot(y_pred, label='Predicción', marker='o')
plt.title('Comparación: Valores Reales vs Predicción')
plt.xlabel('Índice')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.show()

## 6. Pronósticos Futuros

A continuación, se ejemplifica cómo realizar predicciones iterativas para 50 pasos futuros, iniciando desde la última ventana de la serie de prueba.

In [ ]:
future_steps = 50
future_predictions = []

# Comenzamos con la última muestra de X_test
last_sequence = X_test[-1].reshape(1, n_steps, 1)

for _ in range(future_steps):
    # Realiza una predicción
    next_val = model.predict(last_sequence)[0][0]
    future_predictions.append(next_val)
    
    # Desplaza la secuencia e inserta el nuevo valor pronosticado
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_val

plt.figure(figsize=(12, 5))
plt.plot(time_series, label='Serie Original')
plt.plot(
    range(len(time_series), len(time_series) + future_steps), 
    future_predictions,
    'r--', 
    label='Predicciones Futuras'
)
plt.title('Extensión de la Serie con Pronósticos Futuros')
plt.xlabel('Índice')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.show()

## 7. Discusión del Artículo y Conclusiones

### Conexión con el Artículo de Deng et al. (2020)

1. **Captura de Patrones Locales con CNN**: Al igual que en la propuesta de Deng et al. (2020), la arquitectura CNN resulta útil para extraer características locales de la serie, lo que suele mejorar la precisión del pronóstico.
2. **Importancia del Mecanismo de Atención**: El artículo demuestra que incorporar un mecanismo de atención sobre la salida de la CNN (o en capas posteriores) puede enfocar el modelo en los pasos más relevantes de la serie, reduciendo aún más el error. En este notebook no se ha implementado atención, pero su eficacia está respaldada por la literatura.
3. **Mejor Desempeño vs. Métodos Tradicionales**: Según los resultados reportados en el artículo, la CNN con atención supera métodos tradicionales como ARIMA y ML basados en árboles de decisión, especialmente cuando la serie presenta fuertes patrones estacionales o de tendencia.

### Conclusiones y Trabajo Futuro

- El modelo CNN obtuvo un RMSE bajo en el conjunto de prueba, demostrando la capacidad de la convolución para capturar dependencias locales en la serie.
- Los pronósticos futuros muestran la utilidad práctica del modelo, aunque en escenarios de la vida real se recomienda ampliar la ventana de entrenamiento y considerar arquitecturas más complejas.
- Para mejorar la interpretabilidad y la precisión, se puede investigar la integración de mecanismos de atención (tal como propone el artículo) o el uso de enfoques híbridos (CNN + LSTM).
- En aplicaciones reales, se aconseja usar más datos, realizar validaciones exhaustivas y evaluar diferentes horizontes de pronóstico (no solo un paso futuro).

Este ejemplo y la revisión del artículo resaltan el potencial de las CNN en el pronóstico de series temporales. Siguiendo la investigación de Deng et al. (2020), futuras mejoras podrían incluir mecanismos de atención y ajustes específicos para datos reales de la industria eléctrica, donde los patrones son más complejos que en la serie sintética utilizada en este cuaderno.